In [3]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

# ----------------------------
# Configuration
# ----------------------------
base_dir = '/Users/bhaskarpramodchennupalli/Documents/Image_Detector'
csv_path = os.path.join(base_dir, 'train.csv')
train_data_dir = os.path.join(base_dir, 'train_data')
model_path = os.path.join(base_dir, 'model.pth')

# ----------------------------
# Step 1: Read the Original CSV and Randomly Select 5000 Images
# ----------------------------
print("Loading original training CSV...")
df = pd.read_csv(csv_path)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
print(f"Total training samples: {len(df)}")

print("Randomly selecting 5000 images from the training data...")
df_sample = df.sample(n=5000, random_state=42).reset_index(drop=True)
sample_csv_path = os.path.join(base_dir, 'random_5000_train.csv')
df_sample.to_csv(sample_csv_path, index=False)
print(f"Random sample saved to {sample_csv_path}\n")

# ----------------------------
# Step 2: Define the Custom Dataset Class
# ----------------------------
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        print(f"[Dataset] Loaded {len(self.data_frame)} records from {csv_file}")

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # Get the image path and label from the CSV
        file_path = self.data_frame.iloc[idx]['file_name']
        filename = os.path.basename(file_path)
        full_path = os.path.join(train_data_dir, filename)
        # Open the image and convert to RGB
        image = Image.open(full_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.data_frame.iloc[idx]['label']
        return image, label

# ----------------------------
# Step 3: Define Transformations and Create DataLoader
# ----------------------------
print("Defining image transformations...")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # Uncomment the next line if normalization was used during training:
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
print("Transformations defined.\n")

print("Creating dataset from the random sample CSV...")
dataset = CustomImageDataset(csv_file=sample_csv_path, transform=transform)
print(f"Dataset created with {len(dataset)} images.")

batch_size = 32
print(f"Creating DataLoader with batch size {batch_size}...")
# Use num_workers=0 if running in Jupyter; otherwise, adjust as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)
print("DataLoader created.\n")

# ----------------------------
# Step 4: Define the Model Architecture and Load the Trained Model
# ----------------------------
print("Defining the model architecture...")

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # For an input image of size 224x224, one pooling reduces the spatial dimensions to 112x112.
        self.fc_layer = nn.Sequential(
            nn.Linear(16 * 112 * 112, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

print("Instantiating the model...")
model = SimpleCNN()
print("Loading model weights from:", model_path)
model.load_state_dict(torch.load(model_path))
model.eval()
print("Model loaded and set to evaluation mode.\n")

# ----------------------------
# Step 5: Evaluate the Model on the Random Sample
# ----------------------------
print("Running evaluation on the random 5000 training images...")
correct = 0
total = 0

with torch.no_grad():
    for images, labels in dataloader:
        outputs = model(images)
        # Convert logits to probabilities and then to binary predictions (0 or 1)
        preds = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (preds.squeeze() == labels.float()).sum().item()

accuracy = 100 * correct / total
print("\nTest Results on 5000 Random Training Images:")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct: {correct}/{total}\n")

# ----------------------------
# Step 6: Optional – Save Predictions to CSV
# ----------------------------
print("Saving predictions to CSV...")
all_preds = []
with torch.no_grad():
    for images, _ in dataloader:
        outputs = model(images)
        preds = torch.round(torch.sigmoid(outputs))
        all_preds.extend(preds.cpu().numpy().flatten().tolist())

df_sample['prediction'] = all_preds
pred_csv_path = os.path.join(base_dir, 'random_5000_train_predictions.csv')
df_sample.to_csv(pred_csv_path, index=False)
print(f"Predictions saved to '{pred_csv_path}'")


Loading original training CSV...
Total training samples: 79950
Randomly selecting 5000 images from the training data...
Random sample saved to /Users/bhaskarpramodchennupalli/Documents/Image_Detector/random_5000_train.csv

Defining image transformations...
Transformations defined.

Creating dataset from the random sample CSV...
[Dataset] Loaded 5000 records from /Users/bhaskarpramodchennupalli/Documents/Image_Detector/random_5000_train.csv
Dataset created with 5000 images.
Creating DataLoader with batch size 32...
DataLoader created.

Defining the model architecture...
Instantiating the model...
Loading model weights from: /Users/bhaskarpramodchennupalli/Documents/Image_Detector/model.pth
Model loaded and set to evaluation mode.

Running evaluation on the random 5000 training images...

Test Results on 5000 Random Training Images:
Accuracy: 99.46%
Correct: 4973/5000

Saving predictions to CSV...
Predictions saved to '/Users/bhaskarpramodchennupalli/Documents/Image_Detector/random_5000_

In [5]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ----------------------------
# Configuration
# ----------------------------
base_dir = '/Users/bhaskarpramodchennupalli/Documents/Image_Detector'
csv_path = os.path.join(base_dir, 'train.csv')
train_data_dir = os.path.join(base_dir, 'train_data')
model_path = os.path.join(base_dir, 'model.pth')

# ----------------------------
# Step 1: Read the Original CSV
# ----------------------------
print("Loading original training CSV...")
df = pd.read_csv(csv_path)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
total_samples = len(df)
print(f"Total training samples: {total_samples}\n")

# ----------------------------
# Step 2: Define the Custom Dataset Class
# ----------------------------
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        print(f"[Dataset] Loaded {len(self.data_frame)} records from {csv_file}")

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # Get the image path and label from the CSV
        file_path = self.data_frame.iloc[idx]['file_name']
        filename = os.path.basename(file_path)
        full_path = os.path.join(train_data_dir, filename)
        # Open the image and convert to RGB
        image = Image.open(full_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.data_frame.iloc[idx]['label']
        return image, label

# ----------------------------
# Step 3: Define Transformations
# ----------------------------
print("Defining image transformations...")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # Uncomment the following if you used normalization during training:
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
print("Transformations defined.\n")

# ----------------------------
# Step 4: Define the Model Architecture and Load the Trained Model
# ----------------------------
print("Defining the model architecture...")

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # For an image of size 224x224, one pooling reduces dimensions to 112x112.
        self.fc_layer = nn.Sequential(
            nn.Linear(16 * 112 * 112, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

print("Instantiating the model...")
model = SimpleCNN()
print("Loading model weights from:", model_path)
model.load_state_dict(torch.load(model_path))
model.eval()
print("Model loaded and set to evaluation mode.\n")

# ----------------------------
# Step 5: Evaluate on Multiple Sample Sizes
# ----------------------------
# Define the sample sizes you wish to evaluate
sample_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 25000, 40000, 60000, 75000]

# List to keep track of temporary CSV files so we can delete them later
temp_csv_files = []

for sample_size in sample_sizes:
    print(f"=== Evaluating Sample Size: {sample_size} ===")
    if sample_size > total_samples:
        print(f"Requested sample size {sample_size} is greater than total available samples {total_samples}. Skipping...\n")
        continue

    # Randomly sample the requested number of images from the full CSV
    df_sample = df.sample(n=sample_size, random_state=42).reset_index(drop=True)
    temp_csv = os.path.join(base_dir, f"temp_sample_{sample_size}.csv")
    df_sample.to_csv(temp_csv, index=False)
    temp_csv_files.append(temp_csv)
    print(f"Temporary sample CSV saved to: {temp_csv}")

    # Create dataset and DataLoader for the sample
    dataset = CustomImageDataset(csv_file=temp_csv, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)  # Set num_workers=0 for Jupyter

    # Evaluate model on this sample
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            preds = torch.round(torch.sigmoid(outputs))
            total += labels.size(0)
            correct += (preds.squeeze() == labels.float()).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Results on {sample_size} Random Training Images:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Correct: {correct}/{total}\n")

# ----------------------------
# Step 6: Clean Up Temporary Files
# ----------------------------
print("Deleting temporary CSV files...")
for temp_file in temp_csv_files:
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"Deleted {temp_file}")
print("All temporary files deleted.\n")

print("=== Evaluation Process Complete ===")


Loading original training CSV...
Total training samples: 79950

Defining image transformations...
Transformations defined.

Defining the model architecture...
Instantiating the model...
Loading model weights from: /Users/bhaskarpramodchennupalli/Documents/Image_Detector/model.pth
Model loaded and set to evaluation mode.

=== Evaluating Sample Size: 50 ===
Temporary sample CSV saved to: /Users/bhaskarpramodchennupalli/Documents/Image_Detector/temp_sample_50.csv
[Dataset] Loaded 50 records from /Users/bhaskarpramodchennupalli/Documents/Image_Detector/temp_sample_50.csv
Test Results on 50 Random Training Images:
Accuracy: 98.00%
Correct: 49/50

=== Evaluating Sample Size: 100 ===
Temporary sample CSV saved to: /Users/bhaskarpramodchennupalli/Documents/Image_Detector/temp_sample_100.csv
[Dataset] Loaded 100 records from /Users/bhaskarpramodchennupalli/Documents/Image_Detector/temp_sample_100.csv
Test Results on 100 Random Training Images:
Accuracy: 99.00%
Correct: 99/100

=== Evaluating Sa

In [7]:
#Testing ResNet

In [9]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# ---------------------------
# Device Setup
# ---------------------------
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# ---------------------------
# Define Paths
# ---------------------------
base_dir = '/Users/bhaskarpramodchennupalli/Documents/Image_Detector'
test_csv_path = os.path.join(base_dir, 'test.csv')  # CSV containing test image info
test_data_dir = os.path.join(base_dir, 'test_data')  # Directory containing test images
model_path = os.path.join(base_dir, 'best_model.pth')  # Saved best model

# ---------------------------
# Define a Custom Dataset for Test Data
# ---------------------------
class TestImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Expects the CSV file to have a column named 'id' that contains the relative image path.
        """
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        print(f"[TestDataset] Loaded {len(self.df)} records from {csv_file}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Use the 'id' column (since your test CSV contains only this column)
        file_name = self.df.iloc[idx]['id']
        # Construct full path: if the CSV contains a relative path, you can join it with the test_data_dir
        full_path = os.path.join(test_data_dir, os.path.basename(file_name))
        image = Image.open(full_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, file_name

# ---------------------------
# Define Image Transformations
# ---------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # If you used normalization during training, uncomment the following:
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ---------------------------
# Create Test Dataset and DataLoader
# ---------------------------
test_dataset = TestImageDataset(csv_file=test_csv_path, transform=transform)
batch_size = 32
# Use num_workers=0 in Jupyter Notebook to avoid multiprocessing issues
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
print(f"Test dataset size: {len(test_dataset)} images")

# ---------------------------
# Define the Model Architecture
# ---------------------------
# Here, we assume your best model was ResNet18 fine-tuned for binary classification.
def create_resnet18():
    weights = models.ResNet18_Weights.DEFAULT
    model = models.resnet18(weights=weights)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)
    return model

model = create_resnet18()
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()
print("Model loaded and set to evaluation mode.")

# ---------------------------
# Run Inference
# ---------------------------
all_preds = []
all_ids = []

with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        # Convert logits to probabilities using sigmoid, then threshold to obtain binary predictions.
        preds = (torch.sigmoid(outputs) > 0.5).int()
        all_preds.extend(preds.cpu().numpy().flatten().tolist())
        all_ids.extend(file_names)

# ---------------------------
# Save Predictions to CSV
# ---------------------------
submission_df = pd.DataFrame({
    'id': all_ids,
    'label': all_preds
})
submission_csv = os.path.join(base_dir, 'submission.csv')
submission_df.to_csv(submission_csv, index=False)
print(f"Submission saved to {submission_csv}")


Using device: mps
[TestDataset] Loaded 5540 records from /Users/bhaskarpramodchennupalli/Documents/Image_Detector/test.csv
Test dataset size: 5540 images
Model loaded and set to evaluation mode.
Submission saved to /Users/bhaskarpramodchennupalli/Documents/Image_Detector/submission.csv


In [10]:
#Testing Each File Seperately

In [16]:
import os
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms, models

def load_model(model_path, device):
    """
    Loads a ResNet18 model fine-tuned for binary classification.
    """
    # Use the recommended weights argument for torchvision>=0.13.
    weights = models.ResNet18_Weights.DEFAULT
    model = models.resnet18(weights=weights)
    num_ftrs = model.fc.in_features
    # Replace the final fully-connected layer for binary classification.
    model.fc = nn.Linear(num_ftrs, 1)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def predict_image(model, image_path, device):
    """
    Loads an image, applies transformations, and returns the binary prediction
    along with the probability.
    """
    # Define the transformation (adjust normalization if used during training)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # Uncomment the next line if normalization was used during training:
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image: {e}")
        return None, None
    
    # Apply transformation and add batch dimension
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image_tensor)
        # Apply sigmoid to get probability (assuming BCEWithLogitsLoss was used during training)
        prob = torch.sigmoid(output)
        # Get binary prediction: 1 if probability > 0.5 else 0
        pred = (prob > 0.5).int().item()
    
    return pred, prob.item()

# ---------------------------
# Main Execution
# ---------------------------
# Set up the device
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# Hardcode the image path
image_path = "/Users/bhaskarpramodchennupalli/Documents/Image_Detector/test_data/0a14a73f896a46009d545bd2b9960294.jpg"

# Path to your saved model (adjust if necessary)
model_path = "/Users/bhaskarpramodchennupalli/Documents/Image_Detector/best_model.pth"

# Load the model
model = load_model(model_path, device)
print("Model loaded and set to evaluation mode.\n")

# Predict on the hardcoded image
prediction, probability = predict_image(model, image_path, device)
if prediction is not None:
    print(f"Prediction for '{image_path}': {prediction} (Probability: {probability:.4f})")
else:
    print("Prediction failed.")


Using device: mps
Model loaded and set to evaluation mode.

Prediction for '/Users/bhaskarpramodchennupalli/Documents/Image_Detector/test_data/0a14a73f896a46009d545bd2b9960294.jpg': 0 (Probability: 0.0000)
